# Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

**This notebook is inspired by a project of the Machine Learning Engineer Nanodegre program of Udacity.**

### Description
--- 

The project consists of building a CNN-based approach to create a dog breed identification app. Additionally, if the input image consists of a human picture the model will suggest the most resembling breed. If something else will be used as input the app will raise an error.

### App Blocks
--- 

The final application consists of 3 main blocks. A human detector that classifies if the input image is a human. A dog detector that classifies if the image is a dog. Finally, a model that assign the most resembling breed to the image.


### Implementation choices
---

  * Human detector --> Pretrained PyTorch model ([facenet-pytorch](https://github.com/timesler/facenet-pytorch.git)).
  * Dog detector  --> Pre-trained VGG-16 Model trained on [ImageNet](http://www.image-net.org/)
  * Breed classifier --> PyTorch custom model

### Datasets
---
  * [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip) that consist in dog images.
  * [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip) that consist in human faces. 

> **Note: The notebook will automatically download the datasets at runtime.**

### Example of the output
---
![Sample Dog Output](https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/sample_dog_output.png?raw=1)

---
### Table of content

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Application
* [Step 6](#step6): Final tests


In [94]:
!pip install facenet_pytorch

In [95]:
import torch
import numpy as np

In [96]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [97]:
# Set to True if we want to skip the training processes. If the model has to be retrained the proper dataset wil
skip_training = False

<a id='step0'></a>
## Step 0: Import Datasets


In [98]:
# Import datasets
import os
if not os.path.isdir("./dogImages/"):
  !wget -q -O tmp.zip https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip && unzip tmp.zip && rm -f tmp.zip
else:
  print("DogImages data already downloaded")

if not os.path.isdir("./lfw"):
  !wget -q -O tmp.zip https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip && unzip tmp.zip && rm -f tmp.zip
else:
  print("Human faces data already downloaded")


DogImages data already downloaded
Human faces data already downloaded


In [99]:
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("./lfw/*/*"))
dog_files = np.array(glob("./dogImages/*/*/*"))

# print number of images in each dataset
print('There are {} total human images.'.format(len(human_files)))
print('There are {} total dog images.'.format(len(dog_files)))

There are 13233 total human images.
There are 8351 total dog images.


<a id='step1'></a>
## Step 1: Detect Humans

### Write a Human Face Detector

In [100]:
# returns "True" if face is detected in image stored at img_path
import matplotlib.image as img

def face_detector(model, img_path):
    """
        Function that classify if the input image is a human. It does it using 
        a model tha detect faces.
    Args:
        model:
            Model used to detect the human.
        img_path (str):
            Path of the image to classify
            
    Returns:
        classification (Bool):
            True if the image is a human picture, False otherwise.   

    """
    # Read the image form path
    image = img.imread(img_path)

    # Detect face
    face, prob = model(image, return_prob=True)

    return True if face is not None else False
      

### Test the Human Face Detector



In [101]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from facenet_pytorch import MTCNN

# Initialaize face detector
mtcnn = MTCNN(device=device)
# Number of first tested images
n_test_images = 100

human_files_short = human_files[:n_test_images]
dog_files_short = dog_files[:n_test_images]

print("Processing human images")
human = [1 for i in tqdm(range(n_test_images)) if 
         face_detector(mtcnn, human_files_short[i])]

print("Processing dog images")
dog = [1 for i in tqdm(range(n_test_images)) if 
       face_detector(mtcnn, dog_files_short[i])]
        
print('\n The percentage of the first {} images in human_files is: {}%'.format(
    n_test_images, sum(human)/n_test_images * 100))
print('The percentage of the first {} images in dog_files is: {}%'.format(
    n_test_images, sum(dog)/n_test_images * 100))



  3%|▎         | 3/100 [00:00<00:03, 25.10it/s]

Processing human images


  2%|▏         | 2/100 [00:00<00:06, 14.81it/s]

Processing dog images


100%|██████████| 100/100 [00:07<00:00, 12.67it/s]


 The percentage of the first 100 images in human_files is: 100.0%
The percentage of the first 100 images in dog_files is: 18.0%


---
<a id='step2'></a>
## Step 2: Detect Dogs

Pre-trained VGG-16 Model on ImageNet [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

The [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a) shows that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268.



In [102]:
import torchvision.models as models

# define VGG16 model
VGG16 = models.vgg16(pretrained=True).to(device)
VGG16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Dog prediction function

In [103]:
from PIL import Image
import torchvision.transforms as T

def VGG16_predict(net, img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class.
    
    Args:
        net (nn.Module): 
          The VGG16 model used to predict.
        img_path (str): 
          path to an image.
        
    Returns:
        Index corresponding to VGG-16 model's prediction.
    '''
    
    # Create the transformer to input a format accepted from the VGG16
    transform = T.Compose([
        T.Resize(256), 
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # TODO fix normalization
    
    # Load and trasnform input image from path
    img = transform(Image.open(img_path).convert('RGB')).to(device)

    
    
    # Prepare the network
    net.eval()
    with torch.no_grad():
        out = net(img.unsqueeze(0))
    
    
    return int(torch.argmax(out)) # predicted class index

In [104]:
import random
# Test the output of VGG16 predict
def test_VGG16_predict(net):
    
    image = human_files[random.randint(0,100)]
    out = VGG16_predict(net,image)
    assert 0 <= out <= 999, f'The output is not valid: {out}'
    
    print('Success')

test_VGG16_predict(VGG16)

Success


In [105]:
def dog_detector(model, img_path):
    """
        Function that classify if the input image is a dog.
    Args:
        model:
            Model used to detect the human.
        img_path (str):
            Path of the image to classify
            
    Returns:
        classification (Bool):
            True if the image is a dog picture, False otherwise.   

    """
    # Detect dog
    return False if 151 <= VGG16_predict(model, img_path) >= 268 else True

      

### Test the Dog Detector



In [106]:

human = [1 for i in range(n_test_images) if dog_detector(VGG16, human_files_short[i])]
dog = [1 for i in range(n_test_images) if dog_detector(VGG16, dog_files_short[i])]

print('The percentage of the images in human_files_short have a detected dog is {}'.format(sum(human)/n_test_images * 100))
print('The percentage of the images in dog_files_short have a detected dog is {}'.format(sum(dog)/n_test_images * 100))

The percentage of the images in human_files_short have a detected dog is 1.0
The percentage of the images in dog_files_short have a detected dog is 100.0


<a id='step3'></a>
## Step 3: CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/Brittany_02625.jpg?raw=1" width="100"> | <img src="https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/Welsh_springer_spaniel_08203.jpg?raw=1" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/Curly-coated_retriever_03896.jpg?raw=1" width="200"> | <img src="https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/American_water_spaniel_00648.jpg?raw=1" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/Labrador_retriever_06457.jpg?raw=1" width="150"> | <img src="https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/Labrador_retriever_06455.jpg?raw=1" width="240"> | <img src="https://github.com/matteoanelli/Dog_breed_classification/blob/master/images/Labrador_retriever_06449.jpg?raw=1" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!


### Specify Data Loaders for the Dog Dataset
The images have been first resized and then cropped. The chosen dimension was 128. The value has been chosen empirically since it guarantees a balance between the model speed and the final accuracy. Examples of other dimensions tested are 256 and 64. Regarding the data augmentation, during the training, some images are randomly horizontally flipped or/and randomly rotated by a degree up to 15. Thanks to this data augmentation the train dataset size has been increased.

In [107]:
import os
from torchvision import datasets
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

transform = {'train' : T.Compose([
                            T.Resize(128),
                            T.RandomHorizontalFlip(p=0.5),
                            T.RandomRotation(degrees=15),
                            T.CenterCrop(128),
                            T.ToTensor(),
                            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # Imagnet standard
                            ]),
            'valid' : T.Compose([
                            T.Resize(128),
                            T.CenterCrop(128),
                            T.ToTensor(),
                            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                            ]),
            'test' : T.Compose([
                            T.Resize(128),
                            T.CenterCrop(128),
                            T.ToTensor(),
                            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                            ])
            }
    
batch_size = 32

train_set = datasets.ImageFolder('./dogImages/train', transform=transform['train'])
traindataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)

valid_set = datasets.ImageFolder('./dogImages/valid', transform=transform['valid'])
validdataloader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=True, drop_last=True)

test_set = datasets.ImageFolder('./dogImages/test', transform=transform['test'])
testdataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=True)

loaders_scratch = {'train' : traindataloader,
                  'valid' : validdataloader,
                  'test' : testdataloader}

### Model Architecture

In [108]:
import torch.nn as nn
import torch.nn.functional as F

# Print module used during debugging
class Print(nn.Module):
    def __init__(self, message="Inside print layer"):
        super(Print, self).__init__()
        self.message = message

    def forward(self, x):
        print(self.message)
        print(x.shape)
        return x

class Flatten(nn.Module):
  def __init__(self):
    super(Flatten, self).__init__()

  def forward(self, x):
      
      return x.view(x.shape[0], -1)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(nn.Conv2d(3,32,5),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
                                nn.Conv2d(32,64,5),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
                                Flatten(),
                                nn.Linear(29 * 29 * 64, 256),
                                nn.ReLU(),
                                nn.Dropout(0.4),
                                nn.Linear(256, 128),
                                nn.ReLU(),
                                nn.Dropout(0.4),
                                nn.Linear(128, 133), # TODO explain 133
                                )
    
    def forward(self, x):
        
        return self.net(x)

In [109]:
#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net().to(device)
model_scratch

Net(
  (net): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
    (7): Linear(in_features=53824, out_features=256, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.4, inplace=False)
    (10): Linear(in_features=256, out_features=128, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.4, inplace=False)
    (13): Linear(in_features=128, out_features=133, bias=True)
  )
)

In [110]:
def test_Net_shapes():
    net = Net()

    # Feed a batch of images from the training data to test the network
    with torch.no_grad():
        images, labels = iter(traindataloader).next()
        print('Shape of the input tensor:', images.shape)

        y = net(images)
        assert y.shape == torch.Size([traindataloader.batch_size, 133]), "Bad shape of y: y.shape={}".format(y.shape)

    print('Success')

test_Net_shapes()

Shape of the input tensor: torch.Size([32, 3, 128, 128])
Success


__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

**Answer** In the beginning, the main idea was to follow the architecture of [LeNet-5](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf). It is a classical well-known architecture used for image classification. I chose this architecture as a base model since is not heavy and generally effective. To improve the accuracy I decided to increase the number of the channel inside the convolutional layers as well as increase the number of neurons inside the fully connected layers. Moreover, after these improvements, the model was reducing the training loss, however, the validation loss was not decreasing after a few epochs. For this reason, I decided to add dropout layers to regularize the network training process. 

The final model architecture is the following:

In [111]:
model_scratch

Net(
  (net): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
    (7): Linear(in_features=53824, out_features=256, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.4, inplace=False)
    (10): Linear(in_features=256, out_features=128, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.4, inplace=False)
    (13): Linear(in_features=128, out_features=133, bias=True)
  )
)

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [112]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.04)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [113]:
def train(n_epochs, loaders, model, optimizer, criterion, device, save_path, saved_valid=None):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    if saved_valid is None:
        valid_loss_min = np.Inf 
    else: 
        valid_loss_min = saved_valid
    stop = 5
    no_imporve = 0
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to device
            data, target = data.to(device), target.to(device)
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data)
            loss = criterion_scratch(output, target)
            loss.backward()
            optimizer.step()
            ## record the average training loss, using something like
                
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            # if batch_idx == 10:
            #     break
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            data, target = data.to(device), target.to(device)
            ## update the average validation loss
            with torch.no_grad():
                output = model(data)
                loss = criterion_scratch(output, target)
                
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

            # if batch_idx == 10:
            #    break
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Model saved with validation loss error: {}'.format(valid_loss))
            valid_loss_min = valid_loss
            no_imporve = 0
        else:
            no_imporve += 1
            if no_imporve == stop:
                print('Early Stopping! Validation error{}'.format(valid_loss_min))
                break
    
    # return trained model
    return model




In [114]:
# train the model
if not skip_training:
    model_scratch = train(2, loaders_scratch, model_scratch, optimizer_scratch, 
                          criterion_scratch, device, './model_scratch.pt')
    model_scratch.load_state_dict(torch.load('./model_scratch.pt'))
else:
    model_scratch = Net()
    model_scratch.load_state_dict(torch.load('./model_scratch.pt'))
    model_scratch.to(device)

Epoch: 1 	Training Loss: 4.883071 	Validation Loss: 4.849319
Model saved with validation loss error: 4.849318981170654
Epoch: 2 	Training Loss: 4.773460 	Validation Loss: 4.595309
Model saved with validation loss error: 4.595308780670166


### Test the Model


In [115]:
def test(loaders, model, criterion, device):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to device
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, device)

Test Loss: 4.589370


Test Accuracy:  3% (25/832)


---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)


In [116]:
## TODO: Specify data loaders
transform = {'train' : T.Compose([
                            T.Resize(224),
                            T.RandomHorizontalFlip(p=0.5),
                            T.RandomRotation(degrees=15),
                            T.CenterCrop(224),
                            T.ToTensor(),
                            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # Imagnet standard
                            ]),
            'valid' : T.Compose([
                            T.Resize(224),
                            T.CenterCrop(224),
                            T.ToTensor(),
                            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                            ]),
            'test' : T.Compose([
                            T.Resize(224),
                            T.CenterCrop(224),
                            T.ToTensor(),
                            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                            ])
            }

batch_size = 32

train_set = datasets.ImageFolder('./dogImages/train', transform=transform['train'])
traindataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)

valid_set = datasets.ImageFolder('./dogImages/valid', transform=transform['valid'])
validdataloader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=True, drop_last=True)

test_set = datasets.ImageFolder('./dogImages/test', transform=transform['test'])
testdataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=True)

loaders_transfer = {'train' : traindataloader,
                  'valid' : validdataloader,
                  'test' : testdataloader}

Model Architecture


In [117]:
import torchvision.models as models
import torch.nn as nn

model_transfer = models.resnet50(pretrained=True)

# Freeze the layers except for the last one.
num_ftrs = model_transfer.fc.in_features
for param in model_transfer.parameters():
    param.requires_grad = False
model_transfer.fc = nn.Linear(num_ftrs, 133)

model_transfer.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [118]:
model_transfer

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [119]:
def test_resnet_shape():
    net = models.resnet50()
    print(net.fc.in_features)
    net.fc = nn.Linear(net.fc.in_features, 133)
    with torch.no_grad():
        images, labels = iter(traindataloader).next()
        print('Shape of the input tensor:', images.shape)

        y = net(images)
        assert y.shape == torch.Size([traindataloader.batch_size, 133]), "Bad shape of y: y.shape={}".format(y.shape)

    print('Success')

test_resnet_shape()

2048
Shape of the input tensor: torch.Size([32, 3, 224, 224])
Success


__Answer:__ I decided to use resnet50 as transfer learning architecture. The main reason is the high performances that this model has achieved in image classification tasks. Moreover, I have already worked and implemented a Resnet-like architecture in a different academic project. As a result, I was aware of the main concepts behind it. In order to apply the transfer learning, the principal steps have been followed. 
* Load the pre-trained model trained on a large dataset
* Freez all the weights that I do not want to re-train. In our case, all the layers except for the final fully connected layer.
* Replace the final fully connected layer in order to output the expected number of classes (133) 
* Train only the last layer customizing the output for out purposes

### Specify Loss Function and Optimizer


In [120]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.fc.parameters(), lr=0.001, momentum=0.9)

### Train and Validate the Model


In [121]:
# train the model

#model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
# model_transfer.load_state_dict(torch.load('model_transfer.pt'))

if not skip_training:
    model_transfer = train(2, loaders_transfer, model_transfer, optimizer_transfer, 
                          criterion_transfer, device, './model_transfer.pt')
    model_transfer.load_state_dict(torch.load('./model_transfer.pt'))
else:
    model_transfer = models.resnet50()
    model_transfer.fc = nn.Linear(model_transfer.fc.in_features, 133)
    model_transfer.load_state_dict(torch.load('./model_transfer.pt'))
    model_transfer.to(device)

Epoch: 1 	Training Loss: 4.240389 	Validation Loss: 3.387113
Model saved with validation loss error: 3.387113332748413
Epoch: 2 	Training Loss: 2.930423 	Validation Loss: 2.339866
Model saved with validation loss error: 2.3398659229278564


Test the Model

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, device)

### Predict Dog Breed with the Model

In [ ]:
# Preprocess the classes. Example 001.Affenpinscher to Affenpinscher (keeping order)
class_names = [item[4:].replace("_", " ") for item in loaders_transfer['train'].dataset.classes]

# Examples of class names
print("examples of dog breeds \n {}".format(class_names[:20]))

def predict_breed_transfer(net, class_names, img_path):
    # load the image and return the predicted breed
    transform = T.Compose([
        T.Resize(224), 
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    img = transform(Image.open(img_path)).unsqueeze(0).to(device)

    net.eval()
    out = net(img)
    return class_names[torch.argmax(out)]

---
<a id='step5'></a>
## Step 5: Application

The application will accept a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.


In [ ]:
def run_app(dog_model, human_model, breed_classifier, img_path):
    ## handle cases for a human face, dog, and neither
    
    # Plot the image
    img = Image.open(img_path)
    plt.imshow(img)
    plt.show()
    # check if dog
    if dog_detector(dog_model, img_path):
        out = predict_breed_transfer(breed_classifier, class_names, img_path)
        print("Dog! \nThe image is a dog, the predicted breed is {}".format(out))  
    # check if human
    elif face_detector(human_model, img_path):
        out = predict_breed_transfer(breed_classifier, class_names, img_path)
        print("Human. \n You look like a {}".format(out))
    # output error
    else:
        print("Error! The image is not a dog niether a human. Try again with a different image")

---
<a id='step6'></a>
## Step 6: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?


In [ ]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.
from random import randrange

number_of_figure = 3

human_to_predict = [human_files[randrange(len(human_files))] for _ in range(number_of_figure)]

dog_test = np.array(glob("./dogImages/test/*/*"))
dog_to_predict = [dog_test[randrange(len(dog_test))] for _ in range(number_of_figure+1)]

for human, dog in zip(human_to_predict, dog_to_predict):
    run_app(VGG16, mtcnn, model_transfer, human)
    run_app(VGG16, mtcnn, model_transfer, dog)
    
# test labrador 
# labradors = ['images/Labrador_retriever_06457.jpg', 'images/Labrador_retriever_06455.jpg', 'images/Labrador_retriever_06449.jpg']
# for l in labradors:
#     run_app(VGG16, mtcnn, model_transfer, l)

#test extra input
# run_app('images/sample_cnn.png')